In [26]:
import pickle
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pandas as pd

In [27]:
with open('./data/test.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [28]:
test_data=np.array(test_data)
test_x=test_data[:,0]
test_y=test_data[:,1]


In [29]:
with open('./data/train.pkl', 'rb') as f2:
    train_data = pickle.load(f2)

In [30]:
train_data = np.array(train_data)
np.random.shuffle(train_data)
models = np.array_split(train_data, 10)
train_x=[]
train_y=[]
for model_num in range(10):
    train_x.append(models[model_num][:, 0])
    train_y.append(models[model_num][:, 1])
train_x =np.array(train_x)
train_y=np.array(train_y)
bias=[]
vari=[]
err=[]

In [31]:
# outputs
output=[]
#loop through degrees
for degree in range(1,21):
    # loop through models
    output.append([])
    for i in range(10):
        x_train = PolynomialFeatures(degree=degree).fit_transform(train_x[i].reshape(-1,1))
        x_test = PolynomialFeatures(degree=degree).fit_transform(test_x.reshape(-1,1))
        curr_model = LinearRegression().fit(x_train,train_y[i])
        output[degree-1].append(curr_model.predict(x_test))

In [32]:
def bias_var():
    """global bias
    global bias_sq
    global vari
    global bias_per_point
    global bias_sq_per_point
    global vari_per_point
    global bias_per_model
    global bias_sq_per_model
    global vari_per_model
    
    bias = []
    bias_sq = []
    vari = []
    bias_per_point = []
    bias_sq_per_point = []
    vari_per_point = []
    bias_per_model = []
    bias_sq_per_model = []
    vari_per_model = []
    
    actual_f = np.array(test_y)
    exp_actual_f = np.mean(actual_f)
    
    for degree in range(1, 21):
        bias_per_point.append([])
        bias_sq_per_point.append([])
        vari_per_point.append([])
        bias_per_model.append([])
        bias_sq_per_model.append([])
        vari_per_model.append([])
        
        for i in range(10):
            bias_per_point[degree-1].append([])
            bias_sq_per_point[degree-1].append([])
            vari_per_point[degree-1].append([])
            mean_val = np.mean(np.array(output[degree-1][i]))
            
            for j in range(len(output[degree-1][i])):
                bias_per_point[degree-1][i].append(mean_val - actual_f[j])
                bias_sq_per_point[degree-1][i].append((mean_val - actual_f[j])**2)
                vari_per_point[degree-1][i].append((mean_val - output[degree-1][i][j])**2)
            
            bias_per_model[degree-1].append(np.mean(np.array(bias_per_point[degree-1][i])))
            bias_sq_per_model[degree-1].append(np.mean(np.array(bias_sq_per_point[degree-1][i])))
            vari_per_model[degree-1].append(np.mean(np.array(vari_per_point[degree-1][i])))
        
        bias.append(np.mean(np.array(bias_per_model[degree-1])))
        bias_sq.append(np.mean(np.array(bias_sq_per_model[degree-1])))
        vari.append(np.mean(np.array(vari_per_model[degree-1])))"""
        
            
    """for degree in range(1,21):
        output_curr_degree= output[degree-1]
        output_curr_degree=np.array(output_curr_degree)
        mean_point_over_10 = np.mean(output_curr_degree, axis = 0)
        bias.append(np.mean((mean_point_over_10 - test_y)**2))
        var_point_over_10 = np.var(output_curr_degree, axis = 0)
        vari.append(np.mean(var_point_over_10))"""
    global bias
    global vari
    global err
    global test_y
    bias=[]
    vari=[]
    err=[]
    length=len(test_y)
    for degree in range(1,21):
        output_curr_degree = output[degree-1]
        sum_b=0
        sum_v=0
        sum_e=0
        for i in range(10):
            yhat = np.array(output_curr_degree[i])
            exp_yhat= np.array([np.mean(yhat)]*length)
            y=np.array(test_y)
            point_bias_sq=(exp_yhat-y)**2
            mean_bias_sq = np.mean(point_bias_sq)
            varia = np.var(yhat)
            sum_b += mean_bias_sq
            sum_v += varia
            sum_e += np.mean((y-yhat)**2)-mean_bias_sq-varia
        bias.append(sum_b/10)
        vari.append(sum_v/10)
        err.append(sum_e/10)
            
            
    table = pd.DataFrame({"Degree":np.arange(1,21),"Bias square": bias,"Variance": vari, "Error": err})
    print(table.to_string(index=False))
        
    
    
    

In [33]:
bias_var()

 Degree   Bias square      Variance         Error
      1  6.679147e+06  4.791743e+06 -1.043379e+07
      2  6.677429e+06  4.908478e+06 -1.057720e+07
      3  6.629177e+06  6.996113e+06 -1.352475e+07
      4  6.634759e+06  6.964166e+06 -1.346476e+07
      5  6.634102e+06  6.968298e+06 -1.344816e+07
      6  6.634155e+06  6.953843e+06 -1.340026e+07
      7  6.634526e+06  6.921044e+06 -1.333042e+07
      8  6.640312e+06  6.916675e+06 -1.330269e+07
      9  6.641034e+06  6.959338e+06 -1.330503e+07
     10  6.639935e+06  6.923102e+06 -1.329396e+07
     11  6.636181e+06  6.959932e+06 -1.332538e+07
     12  6.637330e+06  6.939830e+06 -1.328432e+07
     13  6.634224e+06  6.989767e+06 -1.333423e+07
     14  6.634624e+06  6.899212e+06 -1.323687e+07
     15  6.635538e+06  6.888943e+06 -1.318866e+07
     16  6.635451e+06  6.884670e+06 -1.317076e+07
     17  6.637243e+06  6.818168e+06 -1.304903e+07
     18  6.637189e+06  6.823189e+06 -1.303341e+07
     19  6.639454e+06  6.713025e+06 -1.285012e+07
